In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('C:/Users/ScMa9004/Documents/RI/RIEXPORT_1105_SYND_M.xlsx')

In [5]:
pr = pd.read_excel('C:/Users/ScMa9004/Documents/RI/panel_report_00001_14.01.2020.xlsx', sheet_name = 1)

In [2]:
sf = pd.read_excel('C:/Users/ScMa9004/Documents/Справочник OPS_регион_QCT_12.2019.xlsx', sheet_name = 'данные')

In [6]:
df.columns

Index(['Store_Type', 'NOT_NEED_CHA', 'NEED_CCODE', 'sms_service', 'RU_CHA',
       'RU_CCN', 'RU_REGION', 'RU_RGN', 'CCN_BEER', 'RGN_BEER', 'CCN_CIG',
       'RGN_CIG', 'REGION_CIG', 'Postcode', 'hl019', 'hl020', 'hl021', 'hl022',
       'hl023', 'hl024', 'hl025', 'hl026', 'hl028', 'hl029', 'hl030',
       'mbr_par', 'STORE_ADDRESS_DA', 'Province', 'City/Regency', 'Store Name',
       'DA_STORE_TYPE', 'Store Chain Name', 'CityCode', 'Trade Type',
       'Store Owner Name', 'Latitude', 'Longitude', 'Sms ID',
       'RMS_CLUSTER_CODE', 'RU_SUBJECT', 'RU_PGR', 'DIST_STRATA', 'Sales Area',
       'hl013', 'hl156', 'autorized', 'Period', 'Set', 'SMPSEQ', 'Index',
       'Index Name', 'Unnamed: 51', 'Cell Name', 'DateofSample', 'StaggingID',
       'GSRID', 'BatchID', 'Stage', 'Period Group', 'Sample Area ID', 'BO_Set',
       'Deficit', 'LOST_SMS_ID', 'LOST_STORE_TYPE', 'SampleAreaName'],
      dtype='object')

In [7]:
def syndi(df, pr, sf):
    
    df = df[df['Stage'] == 'SYNDICATION'].reset_index(drop = True)
    
    df['Cluster'] = np.nan
    df['QCTNAME'] = np.nan
    df['BST'] = np.nan
    df['NEW BST'] = np.nan
    df['CCODE'] = np.nan
    
    pr['outlet10'] = pr['outlet10'].astype('str')
    
    for i in range(len(df['Cluster'])):
        df.loc[i, 'BST'] = pr['bst'][pr['outlet10'] == str(df['Sms ID'][i])].item()
        df.loc[i, 'CCODE'] = pr['citycode'][pr['outlet10'] == str(df['Sms ID'][i])].item()
        df.loc[i, 'Cluster'] = pr['ops'][pr['outlet10'] == str(df['Sms ID'][i])].item()
        if df['Cluster'][i] == 1:
            df.loc[i, 'Cluster'] = pr['cluster'][pr['outlet10'] == str(df['Sms ID'][i])].item()
            if df['Cluster'][i] == 'MSC1':
                df.loc[i, 'QCTNAME'] = 'Olga Timofeeva'
            else:
                df.loc[i, 'QCTNAME'] = 'Svetlana Makarova'
        try:
            df.loc[i, 'QCTNAME'] =  sf['qct_name_NEW'][df['Cluster'][i] == sf['ops']].dropna().unique().item()
        except:
            continue
            
    pt = pd.DataFrame()
    
    pt = df.pivot_table(index= 'Sms ID', columns='Index Name', values='BatchID').reset_index()
    
    df = df.merge(pt, how='inner')
    
    df = df.rename(columns={"RU BEER AUDIT - DUPLICATE": "BEER",
                   "RU CIG AUDIT - DUPLICATE": "CIG",
                   "RU FMCG AUDIT - DUPLICATE": "FMCG",
                   "RU EXXON AUDIT - DUPLICATE": "EXXON"
                   })
    
    df['PMU\QCT'] = "PMU"
    df['Аутлаеры'] = np.nan
    df['Инструкции от'] = np.nan
    df['комментарии'] = np.nan
    
    if 'EXXON' in df.columns:
        df = df[['Sms ID', 'Store Name', 'STORE_ADDRESS_DA', 'DA_STORE_TYPE', 'BST', 'FMCG', 'BEER', 'CIG', 'EXXON', 'NEW BST',
                 'CCODE', 'Cluster', 'QCTNAME', 'PMU\QCT', 'Аутлаеры', 'Инструкции от', 'комментарии']]
    else:
        df = df[['Sms ID', 'Store Name', 'STORE_ADDRESS_DA', 'DA_STORE_TYPE', 'BST', 'FMCG', 'BEER', 'CIG', 'NEW BST',
                 'CCODE', 'Cluster', 'QCTNAME', 'PMU\QCT', 'Аутлаеры', 'Инструкции от', 'комментарии']]
        for i in range(len(df['NEW BST'])):
            if df['BST'][i] == 1:
                df.loc[i, 'NEW BST'] = 1
            if pd.notna(df['FMCG'][i]):
                df.loc[i, 'NEW BST'] = 1
            elif df['BST'][i] == 3 or df['BST'][i] == 4:
                df.loc[i, 'NEW BST'] = 501
            elif df['BST'][i] == 998 and pd.notna(df['BEER'][i]):
                df.loc[i, 'NEW BST'] = 999
            elif df['BST'][i] == 999:
                df.loc[i, 'NEW BST'] = 999
                
    df = df.drop_duplicates(keep = 'first')
                
    return df

In [8]:
syndi(df, pr, sf).to_excel('syndi.xlsx', index = False)

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwi